This notebook is for preparation data before an analysis

In [28]:
import numpy as np
import pandas as pd
import cooler
from cooltools.api.sample import sample
from cooltools.api.insulation import calculate_insulation_score
from cooltools.api.eigdecomp import eigs_cis
import bioframe

from matplotlib import pyplot as plt
import seaborn as sns

from glob import glob
from tqdm.notebook import tqdm
from natsort import natsorted
from os.path import basename, splitext

# Hi-C

In [9]:
FILES

['/tank/projects/kriukov_chromatin/HIC/KO_female_2.mm10.mapq_30.1000.mcool',
 '/tank/projects/kriukov_chromatin/HIC/KO_female_3.mm10.mapq_30.1000.mcool',
 '/tank/projects/kriukov_chromatin/HIC/KO_male_1.mm10.mapq_30.1000.mcool',
 '/tank/projects/kriukov_chromatin/HIC/WT_female_2.mm10.mapq_30.1000.mcool',
 '/tank/projects/kriukov_chromatin/HIC/WT_female_3.mm10.mapq_30.1000.mcool',
 '/tank/projects/kriukov_chromatin/HIC/WT_male_1.mm10.mapq_30.1000.mcool',
 '/tank/projects/kriukov_chromatin/HIC/old_female_2.mm10.mapq_30.1000.mcool',
 '/tank/projects/kriukov_chromatin/HIC/old_female_3.mm10.mapq_30.1000.mcool',
 '/tank/projects/kriukov_chromatin/HIC/old_male_1.mm10.mapq_30.1000.mcool',
 '/tank/projects/kriukov_chromatin/HIC/young_female_2.mm10.mapq_30.1000.mcool',
 '/tank/projects/kriukov_chromatin/HIC/young_female_3.mm10.mapq_30.1000.mcool',
 '/tank/projects/kriukov_chromatin/HIC/young_male_1.mm10.mapq_30.1000.mcool']

In [10]:
#paths
COOLER_FOLDER_PATH = '/tank/projects/kriukov_chromatin/HIC'
DOWNSAMPLED_FOLDER_PATH = '/tank/projects/kriukov_chromatin/HIC/downsampled'
INSULATION_PATH = '/tank/projects/kriukov_chromatin/HIC/insulation'
COMPARTMENTS_PATH = '/tank/projects/kriukov_chromatin/HIC/compartments'
GENOME_FOLDER_PATH = '/tank/projects/kriukov_chromatin/GENOME'

RESOLUTION = 50000
FILES = natsorted(glob('%s/*.mcool' % COOLER_FOLDER_PATH))
#FILES = natsorted(glob('%s/*%s.cool' % (DOWNSAMPLED_FOLDER_PATH, RESOLUTION)))

CLRS = [cooler.Cooler(p + "::/resolutions/%d" % RESOLUTION) for p in FILES]
#CLRS = [cooler.Cooler(p) for p in FILES]
LABELS = [os.path.basename(p).split('.')[0] for p in FILES]

In [12]:
#downsampling procedure
if not os.path.exists(DOWNSAMPLED_FOLDER_PATH):
    os.makedirs(DOWNSAMPLED_FOLDER_PATH)

RESOLUTION = 50_000
MIN_SUM = min([c.info['sum'] for c in CLRS])
for l, c in zip(LABELS, CLRS):
    sample(c, '%s/%s_down_%d.cool' % (DOWNSAMPLED_FOLDER_PATH, l, RESOLUTION), 
                  count=MIN_SUM//1000*1000, 
                  exact=False,
                  chunksize=100_000_000)

INFO:cooler.create:Creating cooler at "/tank/projects/kriukov_chromatin/HIC/downsampled/KO_female_2_down_50000.cool::/"
INFO:cooler.create:Writing chroms
INFO:cooler.create:Writing bins
/home/dkriukov/miniconda3/envs/mice/lib/python3.7/site-packages/cooler/core.py:123: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif is_categorical(data):
INFO:cooler.create:Writing pixels
INFO:cooler.create:Writing indexes
INFO:cooler.create:Writing info
INFO:cooler.create:Creating cooler at "/tank/projects/kriukov_chromatin/HIC/downsampled/KO_female_3_down_50000.cool::/"
INFO:cooler.create:Writing chroms
INFO:cooler.create:Writing bins
INFO:cooler.create:Writing pixels
INFO:cooler.create:Writing indexes
INFO:cooler.create:Writing info
INFO:cooler.create:Creating cooler at "/tank/projects/kriukov_chromatin/HIC/downsampled/KO_male_1_down_50000.cool::/"
INFO:cooler.create:Writing chroms
INFO:cooler.create:Writing bins
INFO:coole

In [15]:
#insulation scores
if not os.path.exists(INSULATION_PATH):
    os.makedirs(INSULATION_PATH)

df = pd.DataFrame()
for l, c in zip(LABELS, CLRS):
    IS = calculate_insulation_score(c, 
                                    window_bp=[RESOLUTION * 10], 
                                    ignore_diags=2, 
                                    append_raw_scores=False)
    df[l] = np.array(IS.filter(regex=("log2*"))).squeeze()


##additional masking around NaN
binstatus = pd.DataFrame({basename(c.filename):c.bins()[:]['weight'] for c in CLRS})

k = 11 #left and right around nan will be nan to avoid `edge effect`
N = df.shape[0]
nanids = binstatus[binstatus.isna().any(1)].index.tolist()
for i in nanids:
    l = i - k if i - k >=0 else 0
    r = i + k if i + k < N else N
    df.iloc[l:r] = np.nan

#postprocessing
IS_info = IS[['chrom', 'start', 'end']].reset_index()
IS_info['index'] = IS_info['index'].apply(str
                                         )
df = df.reset_index()
df['index'] = df['index'].apply(str)
df = df.merge(IS_info, on='index')
df.to_csv('%s/IS_%s.csv' % (INSULATION_PATH, RESOLUTION))

In [18]:
#compartments
if not os.path.exists(COMPARTMENTS_PATH):
    os.makedirs(COMPARTMENTS_PATH)

mm10 = bioframe.fetch_chromsizes('mm10')
chromsizes = bioframe.fetch_chromsizes('mm10')
chromosomes = list(chromsizes.index)

In [19]:
bins = cooler.binnify(mm10, binsize=RESOLUTION)
fasta_records = bioframe.load_fasta('%s/GCF_000001635.27_GRCm39_genomic.fna' % GENOME_FOLDER_PATH)

###CONVERT REFSEQ to CHROMNAMES
fasta_annot = pd.read_csv('%s/GCF_000001635.27_GRCm39_assembly_report.txt' % GENOME_FOLDER_PATH, sep='\t', skiprows=28)
fasta_annot = fasta_annot[fasta_annot['Sequence-Role']=='assembled-molecule']
fasta_keys = fasta_annot[['Assigned-Molecule', 'RefSeq-Accn']].set_index('RefSeq-Accn')
filt_keys = fasta_keys.loc[list(set(list(fasta_records.keys())).intersection(fasta_keys.index))].to_dict()
new_dict = {}
for key, item in fasta_records.items():
    if key in filt_keys['Assigned-Molecule'].keys():
        nk = 'chr' + filt_keys['Assigned-Molecule'][key]
        nk = 'chrM' if nk == 'chrMT' else nk
        new_dict[nk] = item
        
gc = bioframe.frac_gc(bins, new_dict)

#######
bins['GC'] = gc['GC']
bins = bins.drop(bins[bins.chrom=='chrM'].index)

##additional masking around NaN
binstatus = pd.DataFrame({basename(c.filename):c.bins()[:]['weight'] for c in CLRS})
mask_indices = binstatus[binstatus.isna().any(1)].index

In [34]:
lam = {}
eigs = {}

for l,c in zip(LABELS, CLRS):
    lam[l], eigs[l] = eigs_cis(
        c,
        bins=bins,
        phasing_track_col='GC',
        n_eigs=8,
        bad_bins=mask_indices,
        ignore_diags=2,
        sort_metric='var_explained')

    # Save text files
    lam[l].to_csv(f'{COMPARTMENTS_PATH}/{l}_{RESOLUTION}.cis.lam.txt', sep='\t')
    eigs[l].to_csv(f'{COMPARTMENTS_PATH}/{l}_{RESOLUTION}.cis.vecs.txt', sep='\t', index=False)

In [36]:
!ls /tank/projects/kriukov_chromatin/HIC/compartments

KO_female_2_50000.cis.lam.txt	old_female_2_50000.cis.lam.txt
KO_female_2_50000.cis.vecs.txt	old_female_2_50000.cis.vecs.txt
KO_female_3_50000.cis.lam.txt	old_female_3_50000.cis.lam.txt
KO_female_3_50000.cis.vecs.txt	old_female_3_50000.cis.vecs.txt
KO_male_1_50000.cis.lam.txt	old_male_1_50000.cis.lam.txt
KO_male_1_50000.cis.vecs.txt	old_male_1_50000.cis.vecs.txt
WT_female_2_50000.cis.lam.txt	young_female_2_50000.cis.lam.txt
WT_female_2_50000.cis.vecs.txt	young_female_2_50000.cis.vecs.txt
WT_female_3_50000.cis.lam.txt	young_female_3_50000.cis.lam.txt
WT_female_3_50000.cis.vecs.txt	young_female_3_50000.cis.vecs.txt
WT_male_1_50000.cis.lam.txt	young_male_1_50000.cis.lam.txt
WT_male_1_50000.cis.vecs.txt	young_male_1_50000.cis.vecs.txt


# ATAC-seq